# Aviation Database

Cleans Aviation Dataset According to the Needs of Future Implimentation. Current version outputs dataframe as a csv to "../output/CleanAviationData.csv"

Provided by Kheirallah Samaha
<br>
https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
from datetime import datetime

In [2]:
mongo = MongoClient(port=27017)

In [3]:
!mongoimport --type csv -d aviation -c accidents --headerline --drop "..\\data\\AviationData.csv"

!mongoimport --type csv -d aviation -c state_codes --headerline --drop "..\\data\\USState_Codes.csv"

2023-04-17T19:46:33.909-0500	connected to: mongodb://localhost/
2023-04-17T19:46:33.910-0500	dropping: aviation.accidents
2023-04-17T19:46:36.695-0500	88889 document(s) imported successfully. 0 document(s) failed to import.
2023-04-17T19:46:37.356-0500	connected to: mongodb://localhost/
2023-04-17T19:46:37.357-0500	dropping: aviation.state_codes
2023-04-17T19:46:37.381-0500	62 document(s) imported successfully. 0 document(s) failed to import.


In [4]:
db = mongo.aviation
accident_c = db.accidents
state_c = db.state_codes

In [5]:
# change Event.Date to Date codes
accident_c.aggregate([{
    "$project": {
        "Event.Date": {
            "$toDate": "$Event.Date"
            }
        }
    }
])


In [6]:
# exclude flight records without coordinate data

query = {
    "Latitude": {"$ne":""},
    "Longitude": {"$ne":""}
}

limit = 3

fields = {
    "Event.Date": 1,
    "Latitude": 1,
    "Longitude": 1
}

pprint(list(accident_c.find(query, fields).limit(limit)))



[{'Event': {'Date': '1974-08-30'},
  'Latitude': 36.922223,
  'Longitude': -81.878056,
  '_id': ObjectId('643de86967b0dd3f926213f4')},
 {'Event': {'Date': '1979-09-17'},
  'Latitude': 42.445277,
  'Longitude': -70.758333,
  '_id': ObjectId('643de86967b0dd3f926213fb')},
 {'Event': {'Date': '1982-03-16'},
  'Latitude': 30.757778,
  'Longitude': -88.355555,
  '_id': ObjectId('643de86967b0dd3f9262163f')}]


In [7]:
# aggregate all flight data with coordinate data

event_coord = accident_c.find(query, fields)

df = pd.DataFrame(event_coord)

df.head()

,_id,Event,Latitude,Longitude
0,643de86967b0dd3f926213f4,{'Date': '1974-08-30'},36.9222,-81.8781
1,643de86967b0dd3f926213fb,{'Date': '1979-09-17'},42.4453,-70.7583
2,643de86967b0dd3f9262163f,{'Date': '1982-03-16'},30.7578,-88.3556
3,643de86a67b0dd3f92622234,{'Date': '1983-01-08'},46.0411,-120.85
4,643de86a67b0dd3f92622c28,{'Date': '1983-09-09'},48.12,-113.888


In [8]:
# add "Date" column with DateTime Objects

df["Date"]=""

for record in df.iterrows():
    date = record[1]["Event"]["Date"]
    record[1]["Date"] = datetime.strptime(date, "%Y-%m-%d").date()
df


,_id,Event,Latitude,Longitude,Date
0,643de86967b0dd3f926213f4,{'Date': '1974-08-30'},36.9222,-81.8781,1974-08-30
1,643de86967b0dd3f926213fb,{'Date': '1979-09-17'},42.4453,-70.7583,1979-09-17
2,643de86967b0dd3f9262163f,{'Date': '1982-03-16'},30.7578,-88.3556,1982-03-16
3,643de86a67b0dd3f92622234,{'Date': '1983-01-08'},46.0411,-120.85,1983-01-08
4,643de86a67b0dd3f92622c28,{'Date': '1983-09-09'},48.12,-113.888,1983-09-09
...,...,...,...,...,...
34362,643de86c67b0dd3f92636f13,{'Date': '2022-12-13'},047257N,0109280W,2022-12-13
34363,643de86c67b0dd3f92636f17,{'Date': '2022-12-14'},182724N,0066554W,2022-12-14
34364,643de86c67b0dd3f92636f1a,{'Date': '2022-12-15'},373829N,0972635W,2022-12-15
34365,643de86c67b0dd3f92636f1b,{'Date': '2022-12-16'},282825N,0822719W,2022-12-16


In [9]:
# sort by most recent

df = df.sort_values(by="Date", ascending=False)
df

,_id,Event,Latitude,Longitude,Date
34366,643de86c67b0dd3f92636f24,{'Date': '2022-12-26'},341525N,1112021W,2022-12-26
34365,643de86c67b0dd3f92636f1b,{'Date': '2022-12-16'},282825N,0822719W,2022-12-16
34364,643de86c67b0dd3f92636f1a,{'Date': '2022-12-15'},373829N,0972635W,2022-12-15
34363,643de86c67b0dd3f92636f17,{'Date': '2022-12-14'},182724N,0066554W,2022-12-14
34362,643de86c67b0dd3f92636f13,{'Date': '2022-12-13'},047257N,0109280W,2022-12-13
...,...,...,...,...,...
4,643de86a67b0dd3f92622c28,{'Date': '1983-09-09'},48.12,-113.888,1983-09-09
3,643de86a67b0dd3f92622234,{'Date': '1983-01-08'},46.0411,-120.85,1983-01-08
2,643de86967b0dd3f9262163f,{'Date': '1982-03-16'},30.7578,-88.3556,1982-03-16
1,643de86967b0dd3f926213fb,{'Date': '1979-09-17'},42.4453,-70.7583,1979-09-17


In [15]:
# export CleanAviationData.csv

df.to_csv("../output/CleanAviationData.csv")